In [13]:
import pandas 
import json
import re
import config
import math

In [14]:
data = pandas.read_csv('~/Документы/Данные исследования/2fold/Данные.csv', sep=';')
posts = pandas.read_csv('~/Документы/Данные исследования/2fold/Посты.csv', sep=';')
active = pandas.read_csv('~/Документы/Данные исследования/2fold/Активность.csv', sep=';')
diag1 = pandas.read_csv('~/Документы/Данные исследования/2fold/Бэк.csv', sep=',')
diag2 = pandas.read_csv('~/Документы/Данные исследования/2fold/Пятёрка.csv', sep=',')

In [15]:
def template():
    usrs = {}
    for usr in data['ID']:
        usrs[usr] = {}
    for usr in usrs:
        usrs[usr]['Посты'] = []
        usrs[usr]['Диагностика'] = {}
        usrs[usr]['Дополнительно'] = {}
    return usrs

def parse_posts(usrs):
    for i, lnk in enumerate(posts['Ссылка']):
        res = re.search(r'\d+_', lnk)
        pst = {'Текст' : posts['Текст'][i], 
               'Лайков' : str(posts['Лайков'][i]), 
               'Комментариев' : str(posts['Комментариев'][i]), 
               'Репостов' : str(posts['Репостов'][i]), 
               'Просмотров' : str(posts['Просмотров'][i]),
               'Дата' : str(posts['Дата'][i])}
        usr = res.group(0)[:-1]
        usrs[int(usr)]['Посты'].append(pst)

def parse_active(usrs):
    for usr in active['ID']:
        try:
            usrs[usr]['Активность'] = {}
        except KeyError:
            continue
        usrs[usr]['Активность']['Репостов сделанных пользователем'] = 0
        usrs[usr]['Активность']['Лайков на странице'] = 0
        usrs[usr]['Активность']['Лайков на фотографиях'] = 0
        
    for i, usr in enumerate(active['ID']):
        try:
            if active['Тип объекта'][i] == 'Пост' and active['Тип активности'][i] == 'Репост':
                usrs[usr]['Активность']['Репостов сделанных пользователем'] += 1
            elif active['Тип объекта'][i] == 'Пост' and active['Тип активности'][i] == 'Лайк':
                usrs[usr]['Активность']['Лайков на странице'] += 1
            elif active['Тип объекта'][i] == 'Фотография' and active['Тип активности'][i] == 'Лайк':
                usrs[usr]['Активность']['Лайков на фотографиях'] += 1
        except KeyError:
            continue
        
    for usr in active['ID']:
        try:
            usrs[usr]['Активность']['Репостов сделанных пользователем'] = str(usrs[usr]['Активность']['Репостов сделанных пользователем'])
            usrs[usr]['Активность']['Лайков на странице'] = str(usrs[usr]['Активность']['Лайков на странице'])
            usrs[usr]['Активность']['Лайков на фотографиях'] = str(usrs[usr]['Активность']['Лайков на фотографиях'])
        except KeyError:
            continue
        
def parse_data(usrs):
    for i, usr in enumerate(data['ID']):
        usr = int(usr)
        usrs[usr]['Активность'] = {'Друзей' : str(data['Друзей'][i]),
                                   'Подписчиков' : str(data['Подписчиков'][i]),
                                   'Постов на странице': str(data['Постов'][i]),
                                   'Фото альбомов': str(data['Фото альбомов'][i]),
                                   'Видеозаписей': str(data['Видеозаписей'][i]),
                                   'Аудиозаписей': str(data['Аудиозаписей'][i]),
                                   'Фотографий': str(data['Фотографий'][i]),
                                   'Записок': str(data['Записок'][i]),
                                   'Групп': str(data['Групп (Сообществ + Интересных страниц)'][i]),
                                   'Есть ава': data['Есть фотография'][i],
                                   'Можно запостить на стену': data['Можно запостить на стену'][i],
                                   'Можно отправить приглашение в друзья': data['Можно отправить приглашение в друзья'][i],
                                   'Можно написать в лс': data['Можно написать в лс'][i]
                                  }
        
def parse_pd(usrs):
    for i, usr in enumerate(data['ID']):
        usr = int(usr)
        if data['Забанен'][i] == 'Да':
            data['Забанен'][i] = 1
        if type(data['Вторая половинка'][i]) == str:
            data['Вторая половинка'][i] = 1
        if type(data['Skype'][i]) == str:
            data['Skype'][i] = 1
        if type(data['Facebook'][i]) == str:
            data['Facebook'][i] = 1
        if type(data['Twitter'][i]) == str:
            data['Twitter'][i] = 1
        if type(data['LiveJournal'][i]) == str:
            data['LiveJournal'][i] = 1
        if type(data['Instagram'][i]) == str:
            data['Instagram'][i] = 1
        if type(data['Телефон'][i]) == str:
            data['Телефон'][i] = 1
        if type(data['Почта'][i]) == str:
            data['Почта'][i] = 1
        if type(data['Сайт'][i]) == str:
            data['Сайт'][i] = 1
        
        usrs[usr]['Персональные данные'] = {
            'Имя': data['Имя'][i],
            'Фамилия': data['Фамилия'][i],
            'Пол': data['Пол'][i],
            'Забанен': not math.isnan(data['Забанен'][i]),
            'Дата рождения': data['Дата рождения'][i],
            'Возраст': data['Возраст'][i],
            'Страна': data['Страна'][i],
            'Город': data['Город'][i],
            'Родной город': data['Родной город'][i],
            'Статус': data['Статус'][i],
            'Вторая половинка': not math.isnan(data['Вторая половинка'][i]),
            'Семейное положение': data['Семейное положение'][i],
            'Есть дети': data['Есть дети?'][i],
            'Skype': not math.isnan(data['Skype'][i]),
            'Facebook': not math.isnan(data['Facebook'][i]),
            'Twitter': not math.isnan(data['Twitter'][i]),
            'LiveJournal': not math.isnan(data['LiveJournal'][i]),
            'Instagram': not math.isnan(data['Instagram'][i]),
            'Телефон': not math.isnan(data['Телефон'][i]),
            'Почта': not math.isnan(data['Почта'][i]),
            'Сайт': not math.isnan(data['Сайт'][i]),
            'Деятельность': data['Деятельность'][i],
            'Карьера': data['Карьера'][i],
            'Военная служба': data['Военная служба'][i],
            'Школа': data['Школа'][i],
            'ВУЗ': data['ВУЗ'][i],
            'Род занятия': data['Род занятия'][i],
            'О себе': data['О себе'][i],
            'Любимые книги': data['Любимые книги'][i],
            'Политические предпочтения': data['Политические предпочтения'][i],
            'Языки': data['Языки'][i],
            'Мировоззрение': data['Мировоззрение'][i],
            'Источники вдохновения': data['Источники вдохновения'][i],
            'Главное в людях': data['Главное в людях'][i],
            'Главное в жизни': data['Главное в жизни'][i],
            'Отношение к курению': data['Отношение к курению'][i],
            'Отношение к алкоголю': data['Отношение к алкоголю'][i],
            'Любимые цитаты': data['Любимые цитаты'][i],
            'Любимые телепередачи': data['Любимые телепередачи'][i]
        }

def parse_diag1(usrs):
    for i, usr in enumerate(diag1['1']):
        sum = 0
        if type(diag1['5'][i]) == float:
            pass
        else:
            d = json.loads(diag1['5'][i])
            for i in d:
                sum += d[i] 
            usrs[usr]['Диагностика']['Депрессия'] = str(sum - 21)

        
def parse_diag2(usrs):
    for i, usr in enumerate(diag2['1']):
        try:
            usrs[usr]['Диагностика']['Нейротизм'] = str(diag2['3'][i])
            usrs[usr]['Диагностика']['Экстраверсия'] = str(diag2['4'][i])
            usrs[usr]['Диагностика']['Открытость опыту'] = str(diag2['5'][i])
            usrs[usr]['Диагностика']['Готовность к согласию'] = str(diag2['6'][i])
            usrs[usr]['Диагностика']['Сознательность'] = str(diag2['7'][i])
        except KeyError:
            pass
        
def start():
    usrs = template()
    parse_posts(usrs)
    parse_active(usrs)
    parse_data(usrs)
    parse_pd(usrs)
    parse_diag1(usrs)
    parse_diag2(usrs)
    return usrs

In [17]:
with open('time.json', 'w') as f:
    json.dump(start(), f, ensure_ascii=False)

In [18]:
with open(config.PATH_USRS) as f:
    fold1 = json.load(f)

In [19]:
with open('time.json') as f:
    fold2 = json.load(f)

In [20]:
fold2.update(fold1.copy())
with open(config.PATH_USRS, 'w') as f:
    json.dump(fold2, f, ensure_ascii=False)